In [1]:
%load_ext autoreload
%autoreload 2

In [214]:
import keras
import pandas as pd
import os
import importlib

try:
    os.chdir('py')
except FileNotFoundError:
    pass
import small
from small import Splits as S
_ = importlib.reload(small)

In [199]:
handler = small.AbaloneHandler()
df = handler.read()
datasets = handler.split_and_apply_transforms(df)

In [215]:
input_dim = datasets[S.trn]['X'].shape[0]

In [236]:
model = keras.Sequential([
    keras.layers.Dense(8, activation=keras.activations.relu),
    keras.layers.Dense(7, activation=keras.activations.relu),
    keras.layers.Dense(6, activation=keras.activations.relu),
    keras.layers.Dense(5, activation=keras.activations.relu),
    keras.layers.Dense(4, activation=keras.activations.relu),
    keras.layers.Dense(3, activation=keras.activations.relu),
    keras.layers.Dense(2, activation=keras.activations.relu),
    keras.layers.Dense(2, activation=keras.activations.relu)
])
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

In [ ]:
model.fit(datasets[S.trn]['X'], datasets[S.trn]['y'], epochs=200)